### NLTK Name Entity

In [ ]:
import nltk
from nltk import word_tokenize, sent_tokenize
import pandas as pd
from nltk.tag import pos_tag
nltk.download('tagsets')
nltk.help.upenn_tagset('NNP')
nltk.help.upenn_tagset('NN')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
with open("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Pyspark/ReDe/tr.txt", "r", encoding="utf8") as file:
    text = file.read(1000000)

In [ ]:
words = word_tokenize(text)
pos_tags = nltk.pos_tag(words)
chunks = nltk.ne_chunk(pos_tags, binary=True)

In [ ]:
entities =[]
labels =[]
for chunk in chunks:
    if hasattr(chunk,'label'):
        #print(chunk)
        entities.append(' '.join(c[0] for c in chunk))
        labels.append(chunk.label())
        
entities_labels = list(set(zip(entities, labels)))
entities_df = pd.DataFrame(entities_labels)
entities_df.columns = ["Entities","Labels"]
entities_df